# Дообучение Qwen-3-0.6B для генерации анекдотов

В этом ноутбуке мы дообучим модель Qwen-3-0.6B (Base) на датасете из 120 тысяч анекдотов, используя методы SFT (Supervised Fine-Tuning) и LoRA.
Для улучшения качества выборки мы используем **семантический поиск** (Semantic Search) на основе эмбеддингов, чтобы подобрать анекдоты, наиболее подходящие по смыслу к заданным темам (префиксам).

In [1]:
## 1. Подготовка окружения и данных
!pip install -q transformers peft trl bitsandbytes datasets pymorphy2 nltk gensim sentence-transformers

try:
    from google.colab import drive
    drive.mount('/content/drive')
    output_dir = '/content/drive/MyDrive/qwen_jokes_adapter_semantic'
    print("Google Drive mounted successfully.")
except Exception as e:
    print(f"Failed to mount Google Drive: {e}")
    print("Using local storage for checkpoints.")
    output_dir = './qwen_jokes_adapter_semantic'

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 60.6 MB/s eta 0:00:00
Mounted at /content/drive
Google Drive mounted successfully.


In [2]:
import os
import json
import random
import re
import pandas as pd
import numpy as np
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType
from trl import SFTTrainer, SFTConfig
import torch
from sentence_transformers import SentenceTransformer, util
from peft import PeftModel, PeftConfig

# Фиксируем seed для воспроизводимости
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)


### 1.1 Определение префиксов

Определим список префиксов, которые нас интересуют. Мы будем использовать их для семантического поиска релевантных анекдотов.

In [3]:
prefixes_raw = """
1 Идёт мужик по лесу
2 Встречаются два друга
3 Приходят мужик в бар
4 Жена говорит мужу
5 Приходят альфа, бета и гамма в бар
6 Идёт медведь по лесу
7 Приходит мужик к врачу
8 Встречаются русский, американец и немец
9 Идёт по улице девушка
10 Приходит мужик в магазин
11 Еще сто лет назад
12 Встречаются Вовочка и Петька
13 Идёт по лесу охотник
14 Я хорошо готовлю, стираю и убираю в квартире
15 Жена спрашивает у мужа
16 Сидят в баре два друга
17 Идёт по пустыне караван
18 Приходит мужик в аптеку
19 Встречаются два программиста
20 - Послушайте, у этого парня в резюме
21 Приходит мужик в банк
22 Сидят на скамейке два пенсионера
23 Идёт по лесу грибник
24 Приходит мужик в ресторан
25 - Я дочитал учебник по теории вероятности
26 Идёт по улице студент
27 Заходит студент в кофейню
28 Сидят в очереди два человека
29 Идёт по лесу шаман
30 Приходит мужик в библиотеку
31 Идёт по улице кот
32 Встречаются два математика
33 Приходит программист в бар
34 Сидит кот на клавиатуре
35 Доказывает теорему математик
36 Пишет код программист
37 Спрашивает LLM у пользователя
38 Встречаются feature engineer и data scientist
39 Решает уравнение студент
40 Вышел новый альбом Оксимирона
41 Говорит кот хозяину
42 Простой способ остудить чай
43 Доказывает математик теорему
44 Спрашивает математик у кота
45 Пишет промпт для LLM
46 Сидит кот перед монитором
47 Объясняет математик программисту
48 Наняли команду 40 программистов
49 Идёт по крыше кот
50 В статье было написано
51 Спрашивает кот у математика
52 Думает программист о баге
53 Общается пользователь с LLM
54 Сидит кот на книге по алгоритмам
55 Пишет программист тесты
56 Решает LLM задачу по математике
57 Я прочитал книгу Пелевина
58 Встречаются два кота
59 Доказывает программист, что кот - это баг
60 Как часто девушки думают о
61 Примерно двадцать лет назад
62 Классический ML
63 Узнал сегодня забавный факт
64 Я хотел быть самим собой, обычным пацаном
65 За окном шумит Сургут
66 Вообще я люблю только две вещи:
67 Хороший русский рэп
68 Одна бессмысленная ночь у телефона
69 В России запретили
70 Из характеристики:
76 Встречаются overfitting и underfitting
80 Идёт по дому кошка
81 Я из тех людей
82 - Я нормальный.
83 Есть только одна система:
"""

# Обработка списка (удаляем номера и пустые строки)
prefixes = []
for line in prefixes_raw.strip().split('\n'):
    # Удаляем номер в начале (например "1 " или "10 ")
    clean_line = re.sub(r'^\d+\s+', '', line).strip()
    if clean_line:
        prefixes.append(clean_line)

print(f"Загружено {len(prefixes)} префиксов")
print(prefixes[:5])


Загружено 75 префиксов
['Идёт мужик по лесу', 'Встречаются два друга', 'Приходят мужик в бар', 'Жена говорит мужу', 'Приходят альфа, бета и гамма в бар']


### 1.2 Загрузка и семантическая фильтрация датасета

Мы используем модель `cointegrated/rubert-tiny2` для создания векторных представлений (эмбеддингов) анекдотов и префиксов. Затем мы найдем для каждого префикса топ-N наиболее похожих анекдотов по косинусному расстоянию.

In [4]:
import re

def clean_joke(text):
    if not text:
        return text

    # 1. Убираем множественные пробелы и переносы строк
    text = re.sub(r'\s+', ' ', text)

    # 2. Эмодзи (расширенный диапазон — ловит почти все)
    text = re.sub(r'[\U0001F600-\U0001F64F'   # эмоции
                  r'\U0001F300-\U0001F5FF'   # символы, транспорт
                  r'\U0001F680-\U0001F6FF'   # транспорт, знаки
                  r'\U0001F1E0-\U0001F1FF'   # флаги стран
                  r'\U0001F900-\U0001F9FF'   # дополнительные символы
                  r'\U00002600-\U000026FF'   # разные символы (солнце, дождь и т.д.)
                  r'\U00002700-\U000027BF'   # дингбаты
                  r'\U0000FE00-\U0000FE0F'   # вариации селекторы
                  r'\U00002000-\U0000206F'   # общая пунктуация
                  r'\U0001F000-\U0001F02F'   # махджонг, карты
                  r'\U0001F0A0-\U0001F0FF]+', '', text)

    # 3. Хэштеги (#тег, #Тег123)
    text = re.sub(r'#\w+', '', text)

    # 4. комменты (//тег, //Тег123)
    text = re.sub(r'//\w+', '', text)

    # 4. Упоминания (@username, @user_name)
    text = re.sub(r'@\w+', '', text)

    # 5. Ссылки (http, https, t.me, etc.)
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    text = re.sub(r't\.me/\w+', '', text)
    text = re.sub(r'www\.\S+', '', text)

    # 6. Боты и команды (типа /start, /help)
    text = re.sub(r'/\w+', '', text)

    # 7. Лишние знаки в начале/конце (тире, кавычки, скобки и т.д.)
    text = re.sub(r'^[—–\-_\'"«»()[\]{}]+', '', text)
    text = re.sub(r'[—–\-_\'"«»()[\]{}]+$', '', text)

    # 8. Множественные знаки препинания (!!!, ???, ...)
    text = re.sub(r'!{2,}', '!', text)
    text = re.sub(r'\?{2,}', '?', text)
    text = re.sub(r'\.{3,}', '...', text)

    text = re.sub(r'Анекдоты и Шутки StandUp чат истории', '', text)
    text = re.sub(r'Анекдоты от дяди Миши', '', text)

    # 9. Финальная очистка пробелов
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)

    return text

Первый датасет (по итогу не оч удачный, маленький). С ним эксперименты не оч хороошо шли

In [ ]:
!wget -O anek_djvu.txt https://archive.org/download/120_tysyach_anekdotov/anek_djvu.txt


def load_all_jokes(file_path):
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
        text = f.read()

    # Анекдоты в файле разделены пустыми строками
    jokes = text.split('\n\n')

    clean_jokes = []
    for joke in jokes:
        joke = joke.strip()
        # Базовая фильтрация мусора
        if 20 < len(joke) < 1000 and '@' not in joke:
            clean_jokes.append(joke)

    return clean_jokes

dataset_output_dir = os.path.abspath(output_dir+'/..')
dataset_output_path = os.path.join(dataset_output_dir, 'jokes_dataset.json')

try:
    with open(dataset_output_path, 'r', encoding='utf-8') as f:
        selected_jokes = json.load(f)
        print(f"Подгружено {len(selected_jokes)} уникальных семантически близких анекдотов")

except FileNotFoundError:

    all_jokes = load_all_jokes('anek_djvu.txt')
    print(f"Всего загружено {len(all_jokes)} анекдотов")

    # Инициализация модели для эмбеддингов
    embedder = SentenceTransformer('cointegrated/rubert-tiny2')

    # Кодируем все анекдоты (это может занять некоторое время)
    print("Создание эмбеддингов для корпуса анекдотов...")
    corpus_embeddings = embedder.encode(all_jokes, convert_to_tensor=True, show_progress_bar=True)

    # Кодируем префиксы
    print("Создание эмбеддингов для префиксов...")
    query_embeddings = embedder.encode(prefixes, convert_to_tensor=True)

    # Семантический поиск: ищем топ-50 похожих анекдотов для каждого префикса
    top_k = 100
    search_results = util.semantic_search(query_embeddings, corpus_embeddings, top_k=top_k)

    # Собираем уникальные релевантные анекдоты
    relevant_jokes_indices = set()
    for results in search_results:
        for hit in results:
            relevant_jokes_indices.add(hit['corpus_id'])

    selected_jokes = [all_jokes[idx] for idx in relevant_jokes_indices]
    print(f"Отобрано {len(selected_jokes)} уникальных семантически близких анекдотов")

    if len(selected_jokes) > 0:
        print("Пример отобранного анекдота:", selected_jokes[0])

        with open(dataset_output_path, 'w', encoding='utf-8') as f:
            json.dump(selected_jokes, f, ensure_ascii=False, indent=4)


--2025-12-24 16:03:25--  https://archive.org/download/120_tysyach_anekdotov/anek_djvu.txt
Resolving archive.org (archive.org)... 207.241.224.2
Connecting to archive.org (archive.org)|207.241.224.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ia601808.us.archive.org/0/items/120_tysyach_anekdotov/anek_djvu.txt [following]
--2025-12-24 16:03:25--  https://ia601808.us.archive.org/0/items/120_tysyach_anekdotov/anek_djvu.txt
Resolving ia601808.us.archive.org (ia601808.us.archive.org)... 207.241.227.78
Connecting to ia601808.us.archive.org (ia601808.us.archive.org)|207.241.227.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26762684 (26M) [text/plain]
Saving to: ‘anek_djvu.txt’

anek_djvu.txt       100%[===================>]  25.52M  34.2MB/s    in 0.7s    

2025-12-24 16:03:26 (34.2 MB/s) - ‘anek_djvu.txt’ saved [26762684/26762684]

Подгружено 5278 уникальных семантически близких анекдотов


In [ ]:
all_jokes = load_all_jokes('anek_djvu.txt')[:100000]
print(f"Всего загружено {len(all_jokes)} анекдотов")

selected_jokes = all_jokes

Всего загружено 100000 анекдотов


Другой датасет получше

In [4]:
from datasets import load_dataset

# Загружаем датасет (публичный, логина не требует)
ds = load_dataset("samedad/mem-and-russian-jokes-dataset", split="train")

cleaned_dataset_path = os.path.join(output_dir, "clean_russian_jokes.json")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/75.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/521904 [00:00<?, ? examples/s]

In [ ]:
# 1. Загружаем из JSON в список
with open(cleaned_dataset_path, "r", encoding="utf-8") as f:
    jokes = json.load(f)

print(f"Загружено {len(jokes)} анекдотов из файла")
print("Пример:", jokes[0] if jokes else "Пусто")

Очистка датасета

In [6]:
def process_example(example):
    conversations = example["conversations"]
    gpt_message = None
    for msg in conversations:
        if msg["from"] == "gpt":
            gpt_message = msg["value"]
            break

    if gpt_message:
        cleaned = clean_joke(gpt_message)
        if 40 < len(cleaned) < 500:
            return {"joke": cleaned}
    return {"joke": None}

# Загружаем датасет
ds = load_dataset("samedad/mem-and-russian-jokes-dataset", split="train")

# Обрабатываем батчами (batch_size=1000 — оптимально для памяти)
processed_ds = ds.map(
    process_example,
    batched=False,  # по одному, чтобы избежать проблем с None
    num_proc=1,     # 1 процесс, чтобы не жрать память
    remove_columns=ds.column_names  # удаляем старые колонки
)

jokes = [ex["joke"] for ex in processed_ds if ex["joke"] is not None]

print(f"Извлечено {len(jokes)} анекдотов из датасета.")
print("Пример анекдота:", jokes[0] if jokes else "Пусто")

Map:   0%|          | 0/521904 [00:00<?, ? examples/s]

Извлечено 441375 анекдотов из датасета.
Пример анекдота: Стиптизер по кличке Сусанин заводит только поляков


In [7]:
# Сохраняем в файл (на Drive!)
with open(cleaned_dataset_path, "w", encoding="utf-8") as f:
    json.dump(jokes, f, ensure_ascii=False, indent=4)

строим ембединги и семантически фильтруем

In [8]:
embedder = SentenceTransformer('cointegrated/rubert-tiny2')

# Путь для сохранения эмбеддингов
embeddings_path = os.path.join(output_dir, "corpus_embeddings.pt")
indices_path = os.path.join(output_dir, "selected_indices.pkl")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
if os.path.exists(embeddings_path):
    print("Загружаем готовые эмбеддинги...")
    corpus_embeddings = torch.load(embeddings_path)

In [9]:
print("Создание эмбеддингов для корпуса анекдотов...")
corpus_embeddings = embedder.encode(jokes, convert_to_tensor=True, show_progress_bar=True, batch_size=128)
torch.save(corpus_embeddings, embeddings_path)

Создание эмбеддингов для корпуса анекдотов...


Batches:   0%|          | 0/3449 [00:00<?, ?it/s]

In [10]:
print("Создание эмбеддингов для префиксов...")
query_embeddings = embedder.encode(prefixes, convert_to_tensor=True)

print("Семантический поиск...")
top_k = 300
search_results = util.semantic_search(query_embeddings, corpus_embeddings, top_k=top_k)

relevant_jokes_indices = set()
for results in search_results:
    for hit in results:
        relevant_jokes_indices.add(hit['corpus_id'])

selected_jokes = [jokes[idx] for idx in relevant_jokes_indices]
print(f"Отобрано {len(selected_jokes)} уникальных семантически близких анекдотов")

Создание эмбеддингов для префиксов...
Семантический поиск...
Отобрано 15491 уникальных семантически близких анекдотов


In [11]:
# Сохраняем отобранные для будущего
final_jokes_path = os.path.join(output_dir, "selected_jokes_final.json")
with open(final_jokes_path, "w", encoding="utf-8") as f:
    json.dump(selected_jokes, f, ensure_ascii=False, indent=4)

### 1.3 Подготовка данных для SFT

Формируем датасет для обучения. Мы будем использовать ChatML формат, его вроде квен любит

In [12]:
dataset_dir = output_dir

In [ ]:
if os.path.exists(dataset_dir):
    print("Загружаем сохранённый датасет...")
    dataset = load_from_disk(dataset_dir)
    print(f"Загружено {len(dataset)} примеров")
    print("Пример:", dataset[0])
else:
    print("Датасет не найден — нужно создать и сохранить заново")

In [6]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [13]:
def prepare_sft_dataset(jokes):
    data = []
    for joke in jokes:
        if not joke:
            continue

        # Разбиваем на слова (учитываем русские особенности: дефисы, кавычки и т.д.)
        words = joke.split()  # только слова, без пунктуации
        total_words = len(words)

        if total_words < 5:  # слишком короткие пропускаем
            continue

        # Максимум слов для префикса: min(10, 40% от всего)
        max_prefix_words = max(4, min(10, int(total_words * 0.4)))

        # Берём первые max_prefix_words слов
        prefix_words = words[:max_prefix_words]
        prefix = " ".join(prefix_words).strip()
        continuation = joke[len(prefix):].strip()

        # Пропускаем, если продолжение слишком короткое
        if len(continuation.split()) < 3:
            continue

        # Формируем ChatML
        messages = [
            {"role": "system", "content": "Ты генерируешь продолжение анекдота по префиксу."},  # Опционально для стабильности
            {"role": "user", "content": prefix},
            {"role": "assistant", "content": continuation}
        ]

        # Применяем шаблон без токенизации (чтобы сохранить текст)
        formatted_text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False,   # не добавляем начало ответа
            enable_thinking=False          # отключаем <think> режим
        )
        data.append({"text": formatted_text})
    return Dataset.from_pandas(pd.DataFrame(data))

dataset = prepare_sft_dataset(selected_jokes)
print(dataset[0])
print(len(dataset))


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

{'text': '<|im_start|>system\nТы генерируешь продолжение анекдота по префиксу.<|im_end|>\n<|im_start|>user\nХолодно так! Я бы щас дома забралась под плед,<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\nв шерстяных носочках, на кресло с горячим кофе.- Спаришься...- Ага, и спарилась бы с кем-нибудь.<|im_end|>\n'}
14284


In [14]:
dataset.save_to_disk(dataset_dir)

Saving the dataset (0/1 shards):   0%|          | 0/14284 [00:00<?, ? examples/s]

## 2. Загрузка модели и настройка LoRA

In [27]:
base_model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen3-0.6B",
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)
print("Используем Qwen3-0.6B")

last_checkpoint = get_last_checkpoint(output_dir)
last_checkpoint = os.path.join(output_dir, "checkpoint-690")
if last_checkpoint:
    print(f"Загружаем адаптер из {last_checkpoint}")
    model = PeftModel.from_pretrained(base_model, last_checkpoint)
else:
    peft_config = LoraConfig(
        r=16,
        lora_alpha=32,
        lora_dropout=0.1,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
    )

    model = get_peft_model(base_model, peft_config)

model.print_trainable_parameters()
model.gradient_checkpointing_enable()

Используем Qwen3-0.6B
Загружаем адаптер из /content/drive/MyDrive/qwen_jokes_adapter_semantic/checkpoint-690
trainable params: 0 || all params: 606,142,464 || trainable%: 0.0000


/usr/local/lib/python3.12/dist-packages/peft/peft_model.py:598: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight', 'base_model.model.model.layers.0.mlp.gate_proj.lora_A.default.weight', 'base_model.model.model.layers.0.mlp.gate_proj.lora_B.default.weight', 'base_model.model.model.layers.0.mlp.up_proj.lora_A.default.weight', 'base_model.model.model.layers.0.mlp.up_proj.lora_B.default.we

## 3. Обучение (SFT)

In [79]:
from transformers.trainer_utils import get_last_checkpoint

sft_config = SFTConfig(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=1e-5,
    max_grad_norm=0.3,
    max_steps=5000,
    logging_steps=10,
    save_strategy="steps",
    save_steps=30,
    save_total_limit=5,
    fp16=True,
    optim="paged_adamw_8bit",
    report_to="none",
    dataset_text_field="text",
    gradient_checkpointing=True,

    resume_from_checkpoint=last_checkpoint,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    processing_class=tokenizer,
    args=sft_config,
    peft_config=peft_config
)


/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Adding EOS to train dataset:   0%|          | 0/14284 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/14284 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/14284 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


In [31]:
import gc

# памяти gpu t4 не хватало иногда, тут сбрасывается
torch.cuda.empty_cache()
gc.collect()

3527

Собственнно само обучение в несколько прерываний

In [ ]:
if last_checkpoint is not None:
    print(f"Возобновляем с чекпоинта: {last_checkpoint}")

trainer.train(resume_from_checkpoint=last_checkpoint)  # Если None — начнёт с нуля

Возобновляем с чекпоинта: /content/drive/MyDrive/qwen_jokes_adapter_semantic/checkpoint-300


Step,Training Loss
310,1.525400
320,1.536800
330,1.500300
340,1.555400
350,1.525500
360,1.471100
370,1.557600
380,1.496900
390,1.567100
400,1.542300


KeyboardInterrupt: 

In [39]:
if last_checkpoint is not None:
    print(f"Возобновляем с чекпоинта: {last_checkpoint}")

trainer.train(resume_from_checkpoint=last_checkpoint)  # Если None — начнёт с нуля

Возобновляем с чекпоинта: /content/drive/MyDrive/qwen_jokes_adapter_semantic/checkpoint-690


Step,Training Loss
10,3.087400
20,2.558400
30,2.129700
40,2.003300
50,1.933500
60,1.848400
70,1.775000
80,1.691600
90,1.703900
100,1.662900


KeyboardInterrupt: 

In [44]:
last_checkpoint = get_last_checkpoint(output_dir)
if last_checkpoint is not None:
    print(f"Возобновляем с чекпоинта: {last_checkpoint}")

trainer.train(resume_from_checkpoint=last_checkpoint)  # Если None — начнёт с нуля

Возобновляем с чекпоинта: /content/drive/MyDrive/qwen_jokes_adapter_semantic/checkpoint-510


Step,Training Loss
520,1.555000
530,1.497100
540,1.499100
550,1.544000
560,1.505000
570,1.514300
580,1.531800
590,1.487200
600,1.504400
610,1.529900


KeyboardInterrupt: 

In [72]:
last_checkpoint = get_last_checkpoint(output_dir)
if last_checkpoint is not None:
    print(f"Возобновляем с чекпоинта: {last_checkpoint}")

trainer.train(resume_from_checkpoint=os.path.join(output_dir, 'checkpoint-990'))  # Если None — начнёт с нуля

Возобновляем с чекпоинта: /content/drive/MyDrive/qwen_jokes_adapter_semantic/checkpoint-1020


Step,Training Loss
1000,2.430200
1010,1.732000
1020,1.577200
1030,1.503800
1040,1.506500
1050,1.495800
1060,1.529700
1070,1.472700
1080,1.471000
1090,1.452500


KeyboardInterrupt: 

In [75]:
last_checkpoint = get_last_checkpoint(output_dir)
if last_checkpoint is not None:
    print(f"Возобновляем с чекпоинта: {last_checkpoint}")

trainer.train(resume_from_checkpoint=last_checkpoint)  # Если None — начнёт с нуля

Возобновляем с чекпоинта: /content/drive/MyDrive/qwen_jokes_adapter_semantic/checkpoint-1410


Step,Training Loss
1420,1.421200
1430,1.450500
1440,1.461600
1450,1.447500
1460,1.452200
1470,1.458200
1480,1.438700
1490,1.418200
1500,1.449800
1510,1.459700


KeyboardInterrupt: 

In [78]:
last_checkpoint = get_last_checkpoint(output_dir)
if last_checkpoint is not None:
    print(f"Возобновляем с чекпоинта: {last_checkpoint}")

trainer.train(resume_from_checkpoint=last_checkpoint)  # Если None — начнёт с нуля

Возобновляем с чекпоинта: /content/drive/MyDrive/qwen_jokes_adapter_semantic/checkpoint-1770


Step,Training Loss
1780,1.468000
1790,1.440400
1800,1.469600
1810,1.496300
1820,1.445200
1830,1.483100
1840,1.440700
1850,1.453100
1860,1.447800
1870,1.436600


TrainOutput(global_step=2000, training_loss=0.1664827399253845, metrics={'train_runtime': 783.7576, 'train_samples_per_second': 81.658, 'train_steps_per_second': 2.552, 'total_flos': 1.635210559488e+16, 'train_loss': 0.1664827399253845, 'epoch': 4.474936992439092})

In [80]:
last_checkpoint = get_last_checkpoint(output_dir)
if last_checkpoint is not None:
    print(f"Возобновляем с чекпоинта: {last_checkpoint}")

trainer.train(resume_from_checkpoint=os.path.join(output_dir, 'checkpoint-1890'))  # Если None — начнёт с нуля

Возобновляем с чекпоинта: /content/drive/MyDrive/qwen_jokes_adapter_semantic/checkpoint-2000


Step,Training Loss
1900,1.454700
1910,1.429600
1920,1.441100
1930,1.395500
1940,1.408800
1950,1.412800
1960,1.433800
1970,1.395100
1980,1.422400
1990,1.351000


KeyboardInterrupt: 

## 4. Генерация анекдотов

In [8]:
# Загружаем чистую базовую модель
base_model_for_inference = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen3-0.6B",
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

checkpoint_path = os.path.join(output_dir, "checkpoint-2760")

# Накладываем адаптер из чекпоинта
model_inference = PeftModel.from_pretrained(base_model_for_inference, checkpoint_path)

def generate_joke(prefix, model, tokenizer):
    messages = [
        {"role": "system", "content": "Ты генерируешь продолжение анекдота по префиксу."},  # Опционально для стабильности
        {"role": "user", "content": prefix},
    ]

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,    # обязательно: добавляет "<|im_start|>assistant\n"
        enable_thinking=False          # КРИТИЧНО: отключаем thinking mode!
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model_inference.device)

    outputs = model_inference.generate(
        **inputs,
        max_new_tokens=150,
        temperature=0.7,
        top_p=0.9,
        top_k=50,
        repetition_penalty=1.2,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    # Декодируем только сгенерированную часть (без промпта)
    generated_ids = outputs[0][inputs.input_ids.shape[-1]:]
    joke = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()

    return clean_joke(joke)

generated_jokes = []
for i in range(len(prefixes)):
    prefix = prefixes[i]
    joke = generate_joke(prefix, model_inference, tokenizer)
    generated_jokes.append(f"{joke}")


# Сохраняем результаты
generated_jokes_path = os.path.join(output_dir, 'generated_jokes.txt')
prefixes_splited = prefixes_raw.split('\n')[1:-1]

with open(generated_jokes_path, 'w', encoding='utf-8') as f:
    for i in range(len(generated_jokes)):
        joke = generated_jokes[i]
        num = prefixes_splited[i].split()[0].strip()
        f.write(f"{num} {joke}\n")

Такая маленькая модель если и генерит что-то забавное, то не потому что понимает юмор, а потому что не понимает язык, но тем не менее это иногда забавно стреляет